In [98]:
# taken from https://towardsdatascience.com/perceptron-algorithm-in-python-f3ac89d2e537

In [99]:
# imports

from sklearn import datasets
import seaborn as sns
import pandas as pd
import numpy as np

In [100]:
class Generator:

    def __init__ (self):
        self.columns = ['x', 'y', 'labels']

    def makeDF(self):
        return pd.DataFrame(self.makeData(), columns=self.columns)
        
    def makeData(self):
        _x, _y = datasets.make_blobs(
            n_samples = 500,
            n_features = 2,
            centers = 2,
            cluster_std = 1.05,
            random_state = 2)
        return list(zip(_x[:,0], _x[:,1], _y))

In [101]:
class Plotter:

    def plot(self, df):
        sns.scatterplot(
            x = df.columns[0], 
            y = df.columns[1], 
            data = df,
            hue = df.columns[2])

In [102]:
class RandomData: 

    def __init__(self):
        self.df = Generator().makeDF()
        self.plotter = Plotter()

    def plot(self):
        self.plotter.plot(self.generator.df)

In [103]:
class Perceptron:

    def __init__(self, data):
        self.data = data
        self.activation_function = ActivationFunction(0.0)
        self.labels = self.data.iloc[:,-1]
        self.samples = self.data.iloc[:,:-1]
        self.sample_count = self.samples.shape[0]
        self.feature_count  = self.samples.shape[1]
        self.theta = np.zeros((self.feature_count + 1, 1))

    def print_data(self):
        print(f'sample count: {self.sample_count}')
        print(f'feature count: {self.feature_count}\n')
        print(f'samples:\n{self.samples.head()}\n')
        print(f'labels:\n{self.labels.head()}\n')
        print(f'theta:\n{self.theta}\n')


In [104]:
class ActivationFunction:

    def __init__(self, threashold):
        self.threashold = threashold

    def activation_function(self, data):
        return [activate(_x) if _x > self.threashold else deactivate(_x) for _x in self.data]

    def activate(self, data):
        return 1.0

    def deactivate(self, data):
        return 0.0

In [105]:
_data = RandomData()
_perceptron = Perceptron(_data.df)
_perceptron.print_data()

sample count: 500
feature count: 2

samples:
          x          y
0 -1.472648 -10.699273
1 -1.091032  -9.551753
2  1.884512  -1.958476
3 -0.648510  -9.477633
4  0.590249  -0.010609

labels:
0    0
1    0
2    1
3    0
4    1
Name: labels, dtype: int64

theta:
[[0.]
 [0.]
 [0.]]

